In [1]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine("mysql+pymysql://{}:{}@{}/{}?charset={}".format('root', 'welcome2019', '127.0.0.1:3306', 'webdataofteachers','utf8mb4'))
sql = '''
        select * from teachers
      '''
df = pd.read_sql_query(sql, engine)


D:\anaconda\envs\python36\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xD6\\xD0\\xB9\\xFA\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 484")
  result = self._query(query)


In [2]:
# df['email']=df['email'].replace('NaN',"")
zju=df[df['email'].str.contains("zju",na=False)]
sjt=df[df['email'].str.contains("sjtu",na=False)]
hust=df[df['email'].str.contains("hust",na=False)]
ict=df[df['email'].str.contains("ict\.ac",na=False)]
ia=df[df['email'].str.contains("ia\.ac",na=False)]
shanghaitech=df[df['email'].str.contains("shanghaitech",na=False)]
fudan=df[df['email'].str.contains("fudan",na=False)]

In [3]:
stop=['and','the','抽烟','打手机','本科教务管理','个人简介','新加坡','美国','to','for','our','us','(the','a','1.','2.','3.','The','is','be','or','has','it','In','my','from','that','PhD','china','of','in','My','We','also','can','interested','such','are','as','am','this','on','out']
def get_data(y):
    count={}
    for i in y:
        try:
            for j in i:
#                 print(j)
                if j in stop:
                    continue
                if j not in count:
                    count[j]=0
                count[j]=count[j]+1
        except:
            continue
    data=list(count.items())
    return data

In [4]:
import pyecharts.options as opts
from pyecharts.charts import WordCloud
zju['workfield']=zju['workfield'].str.strip()
y=list(zju['workfield'].str.split(' '))
(
    WordCloud(init_opts=opts.InitOpts(width='650px',height='650px'))
    .add(series_name="浙江大学专业词云分析", data_pair=get_data(y), word_size_range=[10,50])
    .set_global_opts(
        
        title_opts=opts.TitleOpts(
            title="浙江大学专业词云分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23),pos_left='center'),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render("./zju_wordcloud.html")
)

D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


'C:\\Users\\Lenovo\\2021-project\\zju_wordcloud.html'

In [5]:
word_list=get_data(y)
args={}
for item in word_list:
    if item[1]>=5:
        args[item[0]]=item[1]

In [6]:
from pyecharts.charts import Pie

x_data = list(args.keys())
y_data = list(args.values())
data_pair = [list(z) for z in zip(x_data, y_data)]
data_pair.sort(key=lambda x: x[1])

(
    Pie(init_opts=opts.InitOpts(width="660px", height="500px", bg_color="#2c343c"))
    .add(
        series_name="访问来源",
        data_pair=data_pair,
        rosetype="radius",
        radius="55%",
        center=["50%", "50%"],
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="Customized Pie",
            pos_left="center",
            pos_top="20",
            title_textstyle_opts=opts.TextStyleOpts(color="#fff"),
        ),
        legend_opts=opts.LegendOpts(is_show=False),
    )
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        label_opts=opts.LabelOpts(color="rgba(255, 255, 255, 0.3)"),
    )
    .render("./customized_pie.html")
)


'C:\\Users\\Lenovo\\2021-project\\customized_pie.html'

In [7]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker

c = (
    Pie()
    .add(
        "",
        [list(z) for z in zip(list(args.keys()), list(args.values()))],
        radius=["40%", "55%"],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="领域-导师人数"))
    .render("./pie_rich_label.html")
)


In [8]:
from pyecharts.charts import Radar

v1 = [list(args.values())]
schema=[]
max_=1000
for x in list(args.keys()):
    schema.append(opts.RadarIndicatorItem(name=x, max_=15))
c = (
    Radar()
    .add_schema(
        schema=schema
    )
    .add("统计", v1)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        legend_opts=opts.LegendOpts(selected_mode="single"),
        title_opts=opts.TitleOpts(title="专业分析雷达图"),
    )
    .render("./专业分析雷达图.html")
)

In [18]:
all={}
for x in word_list:
    if x[1]>2:
        all[x[0]]=x[1]
all

{'云计算': 3,
 '数据挖掘': 8,
 '计算机系统结构': 4,
 '软件工程': 8,
 '大数据分析': 3,
 '人工智能': 15,
 '人机交互': 3,
 '机器学习': 10,
 '计算机网络': 4,
 '计算机应用技术': 7,
 '计算机图形学': 3,
 '金融科技': 3,
 '计算机软件与理论': 6,
 '数据库': 3,
 '大数据智能': 3,
 '物联网': 3,
 '分布式计算': 3,
 '计算机视觉': 5,
 '': 3,
 '计算机应用': 4,
 '数字图书馆': 4}

In [19]:
from pyecharts import options as opts
from pyecharts.charts import Funnel

c = (
    Funnel()
    .add(
        "商品",
        [list(z) for z in zip(list(all.keys()),list(all.values()))],
        sort_="ascending",
#         label_opts=opts.LabelOpts(position="inside"),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Funnel-Sort（ascending）"),legend_opts=opts.LegendOpts(is_show=False))
    .render("./funnel_sort_ascending.html")
)


In [39]:
sjt['workfield']=sjt['workfield'].str.strip()
y=list(sjt['workfield'].str.split(' '))
(
    WordCloud(init_opts=opts.InitOpts(width='650px',height='650px'),)
    .add(series_name="上海交通大学专业词云分析", data_pair=get_data(y), word_size_range=[30,60])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="上海交通大学专业词云分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23),pos_left='center'
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render("./sjt_wordcloud.html")
)

D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'C:\\Users\\Lenovo\\2021-project\\sjt_wordcloud.html'

In [122]:
hust['workfield']=hust['workfield'].str.strip()
y=list(hust['workfield'].str.split(' '))
(
    WordCloud()
    .add(series_name="华中科技大学专业词云分析", data_pair=get_data(y), word_size_range=[15,40])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="华中科技大学专业词云分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23),pos_left='center'
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render("./hust_wordcloud.html")
)

NP难问题高效算法
运筹学
生物信息学
信息哲学
跨专业哲学
FPGA操作系统并行计算计算机体系结构
云计算
云端融合
边缘计算数据中心与绿色计算AI
for
Cloud/Internet
&amp

AI
by
Cloud/Internet软件定义网络与网络虚拟化网络分布式系统
互联网与对等网络
大数据分析社会计算数据挖掘社交网络分析
计算机视觉（人脸识别
人脸活体检测
对象检测（遥感影像中的船只与飞行器检测识别
烟雾火苗检测
行人和车辆检测）
行人属性识别
车辆属性识别
行为分析（奔跑
抽烟
打手机
区域入侵）
拥挤人群计数
烟雾火苗检测）大规模高维数据索引语义哈希深度学习
现代数据库技术
云计算和大数据管理与技术
数据库安全技术等
大数据管理
时空数据库
轨迹数据分析长期招收硕士
博士研究生（可推荐至澳大利亚
美国
新加坡
丹麦等世界知名大学联合培养）欢迎高年级本科生参与科研工作（可推荐至澳大利亚
美国
新加坡
丹麦等世界知名大学攻读博士学位）
实时计算与移动计算深度学习实时与嵌入式系统软件设计
视频编码与图像识别网络流媒体系统与应用认知无线网络与位置服务
云计算
数据中心
网络存储和大数据分析系统
信息存储理论与技术
大数据与机器学习
图像处理与情感识别
大规模数据存储系统
存储可靠性研究
并行分布式计算
基于云存储系统优化研究新型存储系统软件研究基于AI的存储系统优化研究
人工智能(Artificial
Intelligence)
机器学习(Machine
Learning)
深度学习(Deep
Learning)
图像识别（Image
Recognition）
图像分割（Image
Segmentation）
自然语言处理（Natural
LanguageProcessing）等。
嵌入式系统与智能控制
软件工程数据工程
计算机科学理论大数据挖掘人工智能图像处理
现代数据库理论与技术
数据库安全
大数据
机器学习
现代数据库理论与技术
智能控制与嵌入式系统
机器学习
深度学习
实时计算（Real-Time
Computing）
信息物理系统（Cyber
Physical
System）
移动计算（Mobile
Computing）
空间数据管理（Spatial
Data
Management）等
基于机器学习的优化理论与方法研究
求解大规模组合优化问题的

D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'C:\\Users\\Lenovo\\2021-project\\hust_wordcloud.html'

In [91]:
ict['workfield']=ict['workfield'].str.strip()
y=list(ict['workfield'].str.split(' '))
(
    WordCloud()
    .add(series_name="中科院计算所专业词云分析", data_pair=get_data(y), word_size_range=[10,50])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="中科院计算所专业词云分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23),pos_left='center'
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render("./ict_wordcloud.html")
)

智能大数据分析
网络智能化
生物信息学
计算机算法设计与分析
大规模知识处理
多媒体内容安全
虚假信息检测
虚假商品检测
机器学习
计算机视觉
模式识别
高性能计算机体系结构和操作系统
数据管理系统
大数据处理
并行分布式计算
计算机视觉
模式识别
多媒体技术
多模态交互
人工智能
人机交互与普适计算
计算医学
计算机体系结构
机器学习
计算蛋白质组学
无线传感器网络
物联网
网络化微型传感系统
传感器技术
众核处理器设计
高通量处理器设计
数据流处理器设计
编译系统及相关工具
自然语言处理
机器翻译

人机对话
3D几何处理
计算机图形学
三维计算机视觉
深度几何学习
智能路由器
软件定义网络
网络功能虚拟化
信息检索
自然语言理解
大数据分析系统
智能计算系统
计算机体系结构和芯片
智能机器人
智能硬件
数据计算芯片和计算系统
计算机系统结构
机器学习
大数据挖掘等
微处理器设计
计算机系统结构
集成电路设计
计算机体系架构
专用领域架构
自动驾驶感知与决策
神经网络架构搜索
信息编码
多媒体通信网络
存储系统与体系结构
操作系统
分布式系统
多媒体技术
多模态智能
知识工程
知识图谱
社会计算
社会网络
机器学习与数据挖掘
计算机体系结构
网络科学与技术
数字电路设计自动化（EDA）与测试
智能计算
近似计算
多媒体技术
虚拟现实技术
普适计算技术等
程序分析
软件安全
互联网基础资源
大数据分析
网络安全
互联网治理
数字VLSI电路测试及可靠性设计

容错计算
网络传输
网络测量
网络人工智能
计算机网络
移动计算
边缘智能
无人系统网络协同
数据科学
人工智能
异常检测
大图挖掘及系统
计算机体系结构
网络安全
计算机算法
体系结构
并行处理
互联网络
生物信息学
微处理器体系结构
计算机网络
计算机视觉
模式识别
机器学习
人脸识别
情感识别
健康感知
网络数据挖掘
社交网络分析
图神经网络
无线通信
计算与通信相融合
大规模知识处理
知识表示
并行处理体系结构
分布式操作系统
性能评价及文件系统
量子计算
算法复杂性
社会网络近似算法
通信复杂性
判定树复杂性
组合数学
区块链
社交视频
操作系统
虚拟化技术
并行计算机体系结构
高性能计算
多媒体内容分析与检索
计算机视觉与深度学习
模式识别与人工智能
高性能计算机体系结构
处理器微体系结构
并行计

D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'C:\\Users\\Lenovo\\2021-project\\ict_wordcloud.html'

In [107]:
ia['workfield']=ia['workfield'].str.strip()
y=list(ia['workfield'].str.split(' '))
(
    WordCloud()
    .add(series_name="中科院自动化所专业词云分析", data_pair=get_data(y), word_size_range=[25,65])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="中科院自动化所专业词云分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23),pos_left='center'
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render("./ia_wordcloud.html")
)

D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


智能控制
自适应控制
集群智能
智能机器人
：智能理论与技术
1.
智能理论：智能控制与管理
人工智能/混合智能/无人智能
复杂系统平行控制与管理
2.
智能技术：精密检测与控制
动态感知（视频检测
交通遥感）
物联网与边缘计算
云计算与大数据
数字孪生
区块链


3.1

城市智能交通：智能车/无人车/无人机
道路交通
公共交通
轨道交通
综合交通


3.2

智能（制造）企业：3D打印与社会制造
智能机器人
芯片/模组精密封装设备
制造企业管控一体化（ERP/MES/PCS）


3.3

其他：智慧城市


智慧物流
智慧能源等
大数据与深度学习
知识自动化
工业智能技术与系统
智能工厂
智能计算与控制
故障诊断与故障预测
大数据技术及应用
汉字数字化技术
智能控制与计算智能
云计算与大数据分析
智能交通系统
平行交通系统

081101-控制理论与控制工程
智能控制与计算智能
主要包括：计算智能
学习控制
智能控制
最优控制
类脑学习控制
强化学习与深度学习
智能电网
工业控制系统优化
智能控制
鲁棒自适应控制
飞行控制
群体智能
智能控制智能结构
动力学分析与控制
复杂系统的控制机理
个人简介
乔红
中国科学院“****”研究员
博士生导师
国家杰出青年基金获得者
获2014年国家自然科学奖二等奖（第一完成人）
2012年北京市科学技术一等奖（基础研究类
）
2015年北京市科学技术二等奖（技术发明类
）

2013年当选全球IEEE
RAS（机器人与自动化学会）管理委员会委员。

乔红研究员于2004年引进回国并创建中国科学院自动化所“机器人理论与应用”团队（规模50余人）
目前担任中国科学院自动化研究所“类脑智能研究中心”副主任（负责类脑神经机器人方向
是中心三个方向之一）
中国科学院大学计算机与控制学院“控制理论与控制工程”教研室主任
中国科学院自动化研究所-中国科学技术大学联合实验室副主任
中国科学院“脑科学与智能技术卓越中心”核心骨干（机器人方向唯一核心骨干）
九三学社中央科技委委员。

乔红研究员受邀担任国际SCI期刊Assembly
Automation主编
并担任多种著名国际SCI期刊编委
如IEEE
Trans.
Cybernetics
IEEE
Trans.
Automation
Science
and
Engineering

I

协议模型自动生成
正在整合中科院
社科院
清华大学
人民大学
南开大学
中国信保等优质资源
定制国家风险评估模型
产业研究模型和国际舆情分析模型
打造国家级海外投资风险智能检测
识别
预测
预警平台
用智能识别技术快速提升中国智库的智能化水平
快速提升走出去企业的风险管控水平。
个人介绍
兴军亮
2012年毕业于清华大学计算机科学与技术系
获工学博士学位
同年进入中国科学院自动化研究所模式识别国家重点实验室工作
现为中科院自动化所研究员
博士研究生导师
特聘青年骨干。2012年12月至2013年12月期间新加坡国立大学访问研究员
2015年4月至11月微软亚洲研究院网络媒体组“铸星计划”访问教授
美国电器与电子工程学会（IEEE）高级会员
中国计算机学会（CCF）会员
计算机视觉专委会委员。为计算机视觉
机器学习和人工智能。目前已在重要国际期刊如TPAMI
IJCV
TIP
TOMM以及重要国际会议上如ICCV
CVPR
ECCV
ACM
Multimedia
AAAI上发表论文100多篇
出版计算机视觉译著3部
撰写深度学习及人工智能领域著作2部。在攻读博士期间
曾获得清华大学计算机系学术新秀
清华大学优秀博士毕业论文
谷歌学者
清华大学综合一等奖学金等荣誉称号和奖励
在博士后研究期间
带领学生完成的论文“Wow！You
Are
so
Beautiful
Today！”获得了2013年多媒体领域重要国际会议ACM
Multimedia最佳论文奖。多次获得相关研究方向的国际和国内挑战赛冠军。目前作为项目负责人正在主持国家自然科学基金面上项目一项
作为项目实际负责人和核心骨干参与了973和863等多项国家重要研究课题
以及与惠普
英特尔
欧姆龙
富士通
华为等知名企业的多项合作研究课题。正在承担的研究课题主要包括人脸和人体视觉
网络敏感内容安全和大数据分析应用
关心的研究问题包括物体检测
跟踪和分割
图像识别检索以及大数据分析与建模等
关心的技术模型包括Boosting
支持向量机
稀疏表示和深度学习等。近年来重点围绕深度强化学习展开了深入的研究
在智能博弈等人工智能最前沿方向进行探索和研究。
三维计算机视觉
模式识别
：计算机视觉
图像处理
机器学习等。
寄语学生：我们热忱邀请有志于科学研究和高端技术开发的学生加入我们的研究团队
目前
我们团队有一名研究员
一名副研

'C:\\Users\\Lenovo\\2021-project\\ia_wordcloud.html'

In [112]:
shanghaitech['workfield']=shanghaitech['workfield'].str.strip()
y=list(shanghaitech['workfield'].str.split(' '))
(
    WordCloud()
    .add(series_name="上海科技大学专业词云分析", data_pair=get_data(y), word_size_range=[15,60])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="上海科技大学专业词云分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23),pos_left='center'
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render("./shanghaitech_wordcloud.html")
)

网络编码理论
经典信息论
量子信息论
新型超声成像方法
深度学习在超声成像中的应用
超声治疗及新型医疗器械开发
雪崩光电二极管
高速光电探测器
硅基光电子器件
下一代光通信器件
并行
分布式计算
算法和复杂度下界
大数据
机器学习
节能计算
电力电子相关领域：无线电能传输
高频磁集成
超高频变换器
谐振变换器的建模与控制
宽禁带器件的应用
泛在电力物联网
基于人工智能和大数据智慧能源网络等
新型混合模态成像技术:
光声成像
医学传感与成像电路与系统设计
深度学习与计算机视觉在新兴医学领域的应用
计算机视觉和机器学习
FPGA结构
设计工具及应用
超低功耗数字电路与系统设计
硬件安全
智能与无人驾驶汽车
机器学习相关嵌入式系统研究及应用
计算机视觉：场景理解
语义分割
物体检测
视觉与语言
机器学习：概率图模型
神经网络
弱监督和元学习
近似推理
Optimal
Control

优化控制
Model
Predictive
Control

模型预测控制
Robust
Optimization

鲁棒优化
模型降阶
不确定性分析
反问题和深度学习在图像处理中的应用
智能医疗仪器
智能手术辅助系统
基于生理模型的诊断辅助
认知模型在智能驾驶中的应用
计算机视觉
移动系统的三维立体视觉系统
运动恢复结构
视觉定位和匹配
代数几何
微纳电子器件与电路设计
自旋电子学
拓扑量子体系应用
低温电子学
大数据可视化及可视分析
数据科学
可解释性人工智能
数据故事
人机交互
计算社会科学
无线通信
信号处理
压缩感知
机器学习
微能耗嵌入式系统
智能泛在物联网
功率管理集成电路
微能发电技术
机电一体化
模型化简
不确定性量化
大数据算法
偏微分方程数值解
有限元方法
区域分解方法
电磁场理论
超大规模天线阵列与系统
微波
毫米波
太赫兹器件
雷达感知
电磁超材料
机器人微纳操作
柔性传感器
超声全息
人机交互
一：机器人微纳操作及其在生物医学工程
神经科学中的应用。主要的研究内容包括开发微纳尺度的精密检测与驱动技术
末端执行器
以及相应的控制方法等
二：基于MEMS技术
soft-lithography技术
以及
micro-printing技术开发应用于医疗机器人
康复机器人
增强现实等的可穿戴柔性传感器
实现诸如姿态
压力等交互性信息的可靠检测
三：超声全息类似于光全息技术


D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'C:\\Users\\Lenovo\\2021-project\\shanghaitech_wordcloud.html'

In [116]:
fudan['workfield']=fudan['workfield'].str.strip()
y=list(fudan['workfield'].str.split(' '))
(
    WordCloud()
    .add(series_name="复旦大学专业词云分析", data_pair=get_data(y), word_size_range=[15,50])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="复旦大学专业词云分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23),pos_left='center'
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render("./fudan_wordcloud.html")
)

金融科技与安全方向
金融科技与安全方向
数据驱动安全
访问控制
多媒体大数据（视频
图像
音频）分析与检索
深度学习
计算机视觉
模式识别
编码与信息论
密码学与信息安全
计算复杂性
软件维护与演化
自适应软件
移动计算与云计算
自然语言处理
深度学习
大数据并行式分析
数据系统安全及私密
时空移动数据分析
恶意代码检测
漏洞分析挖掘
AI安全
区块链安全和Web安全等
软件工程
企业应用集成
电子商务
多媒体
Web
数据管理
机器视觉与智能计算
数据学与数据科学
体系结构
编译优化和并行计算
密码学
网络与信息安全
计算理论
电子政务
大数据
计算机网络
分布式系统
网络多媒体
密码学
信息安全
网络互动娱乐
VR/AR
数据可视分析
智能人机交互
人机交互与服务工程
电子商务与电子政务
软件工程
软件工程
并行算法与数据结构
系统软件
并行算法与数据结构
系统软件
软件工程
电子商务推荐系统
社会网络
商务智能
大数据分析应用
商务智能
大数据分析应用
机器学习
自然语言处理
智能系统
理论计算机科学
移动互联网
电子商务
网络与信息安全
数据挖掘
软件工程
游戏化
软件工程
媒体计算
电子商务
数字环保

移动流媒体
企业内部控制
分布式系统
数据挖掘
学生工作负责人
网络与信息安全
系统结构
算法设计和分析
网络与信息安全
财务
在职工程硕士教务
院办综合管理
财务
财务
MSE管理
MSE管理
本科教务管理
本科教务管理


D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'C:\\Users\\Lenovo\\2021-project\\fudan_wordcloud.html'